In [7]:
import pandas as pd

df = pd.read_csv('../data/creditcard.csv/cleaned_creditcard.csv')
df.columns #notice how this is the new clean

Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount',
       'Class', 'Amount_scaled'],
      dtype='object')

Apply SMOTE

#Split the data into training and testing sets